In [122]:
from gensim.models.keyedvectors import KeyedVectors
import gensim
import pandas as pn
from nltk import word_tokenize
import numpy as np
import sys
sys.path.append('C:/Users/Juani/chatbot/')
import preprocesamiento
import torch
print(torch.__version__)
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torchvision import datasets
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

0.4.1


In [2]:
wordvectors_file_vec = 'C:/Users/Juani/chatbot/fasttext-sbwc.3.6.e20.vec'
cantidad = 100000
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)


In [3]:
#dataset = pn.read_csv("C:/Users/Juani/chatbot/preprocessedQuestions.csv",header=None,delimiter=',')
correctedData = pn.read_csv("C:/Users/Juani/chatbot/preprocessedQuestions.csv",delimiter=',')
#labels = dataset.values[:,0]
labels = correctedData.values[:,0]
print(correctedData.shape)
#cantidadLabels = dataset.values[len(dataset.values)-1,0] + 1
cantidadLabels = correctedData.values[len(correctedData.values)-1,0] + 1
print(cantidadLabels)
print(labels)
correctedData = correctedData.values

(1487, 2)
103
[0 0 0 ... 102 102 102]


In [4]:
#ESTO NO LO HAGO MÁS PORQUE YA TENGO EL ARCHIVO PREPROCESADO CARGADO

#print(dataset.values)
#correctedData = preprocesamiento.preprocesar(dataset.values,1) #Dataset lematizado
#print(correctedData)
print(type(correctedData))

<class 'numpy.ndarray'>


In [4]:
def contarClase(dataset,clase):
    contador = 0
    for i in range(dataset.shape[0]):
        if dataset[i,0]==clase:
            contador+=1
    return contador

cantidad_preg = correctedData.shape[0]
#minPreguntasPorClase = 2
clase = -1
indxTest = []
indxTrain= []
for i in range(cantidad_preg):
    if correctedData[i,0]!=clase:
        clase = correctedData[i,0]
        cantidadPregClase = contarClase(correctedData,clase)
        #print(clase)
        if clase!=46:
            indxTest.append(np.random.randint(0, cantidadPregClase-1)+i)

#print(indxTest)        
#Separo el dataset en test y train, ya tengo los indices de test (indxTest)
#Obtengo los indices de train

indxTrain = list(filter(lambda x: x not in indxTest, range(cantidad_preg)))        



In [111]:
cantTest = len(indxTest) #102
cantTrain = len(indxTrain) #1385
print(cantidad_preg)


trainY = np.zeros((cantTrain,1), dtype=np.int64)
trainXtext = np.zeros((cantTrain,1), dtype=object)
#trainY = np.zeros((cantTrain,cantidadLabels),dtype=np.int64)
y = np.zeros((cantidadLabels))
testY = np.zeros((cantTest,1), dtype=np.int64)
#testY = np.zeros((cantTest,cantidadLabels),dtype=np.int64)
testXtext = np.zeros((cantTest,1), dtype=object)

contTrain = 0
contTest = 0

print("Tamaño de trainY: ",trainY.shape)

for i in range(cantidad_preg):
    
    if i in indxTest:
        #y[labels[i]] = 1
        #testY[contTest] = y
        #y[labels[i]] = 0
        testY[contTest] = correctedData[i,0]
        testXtext[contTest] = correctedData[i,1]
        contTest+=1
    
    else:
        #y[labels[i]] = 1
        #trainY[contTrain] = y
        #y[labels[i]] = 0
        trainY[contTrain] = correctedData[i,0]
        trainXtext[contTrain] = correctedData[i,1]
        contTrain+=1

#print(trainY)
trainY = torch.from_numpy(trainY)
#trainY = trainY.float()
#trainY = trainY.type(torch.FloatTensor)
testY = torch.from_numpy(testY)
#testY = testY.float()
#testY = testY.type(torch.FloatTensor)

#Acá hago eso de meterle la etiqueta y la magia esa.
#Genero el vector de 0s

#for i in range(cantidad_preg):
#    print(i)
#    y[labels[i]] = 1
#    Y[i,:] = y
#    y[labels[i]] = 0
        

1487
Tamaño de trainY:  (1385, 1)


In [112]:
#CREO LA MATRIZ DE PROMEDIOS PARA trainX

token_idx = 0
wordSet = {}
tipoEmbed = 1

#embeddingMat = np.array([]) #Esto es un tensor... más no! Una besha lista, luchi pedazo de manco
embeddingMat = []
#print(type(embeddingMat))
#embeddingMat = np.array((0,300))
promMat = np.zeros((300,cantTrain),dtype=np.float32)
for i in range(cantTrain):
    words = word_tokenize(trainXtext[i,0])
    words_in_sentence = []
    #prom = np.zeros((300, 1))
    prom = np.zeros((300))
    #prom = np.zeros((1,300))
    for w in words: #w is token
        if w not in wordSet: #la palabra no se encuentra en el diccionario generado: se debe agregar
            #print(w)
            wordSet[w] = token_idx
            if w in wordvectors:
                words_in_sentence.append(token_idx)
                #print("Encontró ",w)
                s = wordvectors.get_vector(w)
                #s = np.reshape(s,(1,300))
                #print(s.shape)
                embeddingMat.append(s) #Acá hay que meter al tensor en verdad. -> precision en los decimales
                #np.append(embeddingMat,s)
                #embeddingMat[cont] = s
                token_idx += 1
                #print(w)
            else:
                #print("No encontró ",w)
                #embeddingMat.append(np.random.randn(300,1))
                embeddingMat.append(np.random.normal(0,0.2,300))
                #embeddingMat.append(np.ones((300,1)))
                #print(token_idx)
                token_idx+=1
                #print(w)
        else: #la palabra ya se encuentra en el diccionario generado: debemos asignarle a la lista de tokens el indice corresp
            words_in_sentence.append(wordSet[w])
    #print(len(embeddingMat))
    #print(words_in_sentence)
    for indx in words_in_sentence: #promediamos solo los vectores de embedding encontrados en el vocabulario original
        #print(indx)
        #print(prom.shape)
        #print(embeddingMat[indx])
        prom += embeddingMat[indx] #Calcular promedio con embeddingMat y los indices de words_in_sentence
    if len(words_in_sentence)>0:
        #print(len(words_in_sentence))
        promMat[:,i] = prom/len(words_in_sentence)

#numpyEmbeddingMat = np.array(embeddingMat)
#print(type(numpyEmbeddingMat))
#tensorEmbedding = torch.from_numpy(embeddingMat)
#tensorEmbedding = torch.as_tensor(embeddingMat)

tensorEmbeddingTrain = torch.Tensor(embeddingMat)
promMat = np.transpose(promMat)
trainX = torch.from_numpy(promMat)
trainX = trainX.float()
#trainX = trainX.type(torch.FloatTensor)
#print(token_idx)
print("Size de tensor embedding: ",tensorEmbeddingTrain.shape)
print("Size de tensor prom: ", tensorProm.shape)
#print(tensorEmbedding[5,:])
#print(tensorEmbedding[0,:])
#print(wordvectors.get_vector('extranjero'))


Size de tensor embedding:  torch.Size([648, 300])
Size de tensor prom:  torch.Size([1385, 300])


In [113]:
#CREO LA MATRIZ DE PROMEDIOS PARA testX

token_idx = 0
wordSet = {}
tipoEmbed = 1

#embeddingMat = np.array([]) #Esto es un tensor... más no! Una besha lista, luchi pedazo de manco
embeddingMat = []
#print(type(embeddingMat))
#embeddingMat = np.array((0,300))
promMat = np.zeros((300,cantTest),dtype = np.float32)
for i in range(cantTest):
    words = word_tokenize(testXtext[i,0])
    words_in_sentence = []
    #prom = np.zeros((300, 1))
    prom = np.zeros((300))
    #prom = np.zeros((1,300))
    for w in words: #w is token
        if w not in wordSet: #la palabra no se encuentra en el diccionario generado: se debe agregar
            #print(w)
            wordSet[w] = token_idx
            if w in wordvectors:
                words_in_sentence.append(token_idx)
                #print("Encontró ",w)
                s = wordvectors.get_vector(w)
                #s = np.reshape(s,(1,300))
                #print(s.shape)
                embeddingMat.append(s) #Acá hay que meter al tensor en verdad. -> precision en los decimales
                #np.append(embeddingMat,s)
                #embeddingMat[cont] = s
                token_idx += 1
                #print(w)
            else:
                #print("No encontró ",w)
                #embeddingMat.append(np.random.randn(300,1))
                embeddingMat.append(np.random.normal(0,0.2,300))
                #embeddingMat.append(np.ones((300,1)))
                #print(token_idx)
                token_idx+=1
                #print(w)
        else: #la palabra ya se encuentra en el diccionario generado: debemos asignarle a la lista de tokens el indice corresp
            words_in_sentence.append(wordSet[w])
    #print(len(embeddingMat))
    #print(words_in_sentence)
    for indx in words_in_sentence: #promediamos solo los vectores de embedding encontrados en el vocabulario original
        #print(indx)
        #print(prom.shape)
        #print(embeddingMat[indx])
        prom += embeddingMat[indx] #Calcular promedio con embeddingMat y los indices de words_in_sentence
    if len(words_in_sentence)>0:
        #print(len(words_in_sentence))
        promMat[:,i] = prom/len(words_in_sentence)

#numpyEmbeddingMat = np.array(embeddingMat)
#print(type(numpyEmbeddingMat))
#tensorEmbedding = torch.from_numpy(embeddingMat)
#tensorEmbedding = torch.as_tensor(embeddingMat)

tensorEmbeddingTest = torch.Tensor(embeddingMat)
promMat = np.transpose(promMat)
testX = torch.from_numpy(promMat)
testX = testX.float()
#testX = testX.type(torch.FloatTensor)
#print(token_idx)
print("Size de tensor embedding: ",tensorEmbeddingTest.shape)
print("Size de tensor prom: ", testX.shape)
print("Tipo del tensor: ",type(testX))
#print(tensorEmbedding[5,:])
#print(tensorEmbedding[0,:])
#print(wordvectors.get_vector('extranjero'))

Size de tensor embedding:  torch.Size([213, 300])
Size de tensor prom:  torch.Size([102, 300])
Tipo del tensor:  <class 'torch.Tensor'>


# Entrenamiento

Tengo que generarme estructuras que permitan entrenar!

Creo los conjuntos de entrenamiento y prueba...

In [114]:
#Creo el trainset y el testset

trainset = torch.utils.data.TensorDataset(trainX,trainY) 
testset = torch.utils.data.TensorDataset(testX,testY)
print(trainset)

#Creo el dataloader

batch_size = 20

trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True) 
testloader = torch.utils.data.DataLoader(testset,batch_size=cantTest,shuffle=True)



Ahora defino la arquitectura del modelo....

In [115]:
class MLP(nn.Module):
    def __init__(self,D_in,H,D_out):
        #constructor: aca se instancian dos modulos del tipo nn.Linear
        #y las asignamos como variables linear1 y linear 2
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(D_in,H)
        self.linear2 = nn.Linear(H,D_out)

    def forward(self, x):
        #definicion del paso hacia adelante, entra un tensor y devuelve otro tensor 
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        #y_pred = F.softmax(self.linear2(h_relu)) #Aplico la softmax 
        
        return y_pred
    

Instancio el modelo....

In [116]:
prueba2 = MLP(300,10,cantidadLabels) #Neuronas de entrada, D_in, D_out

Defino la función de pérdida e instancio el optimizador..

In [117]:
criterion = nn.CrossEntropyLoss(reduction = "sum")
optimizerPrueba2 = optim.Adam(prueba2.parameters())

Inicializo un vector para la func. de pérdida y arranco a entrenar...

In [121]:
lossVec = []
ejex = []
epoca = 50

print("------- Entrenando un MLP con 300 parámetros y 10 neuronas en la capa oculta -------------" )
for t in range(epoca):  # 500 #epocas
    for batch_idx, (XX, YY) in enumerate(trainloader):
        out = prueba2(XX)
        #print(YY.shape)
        (tam) = YY.size()
        YY = YY.view(tam[0])
        #print(out.dtype)
        #print(YY.dtype)
        loss = criterion(out, YY)
        optimizerPrueba2.zero_grad()
        loss.backward()
        optimizerPrueba2.step()
    lossVec.append(loss)
    print("Epoca MLP optimizador ADAM: ", (t,loss.item()))
    ejex.append(t)
    
print("----Terminó el entrenamiento----")
print(lossVec)

------- Entrenando un MLP con 300 parámetros y 10 neuronas en la capa oculta -------------
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Siz

torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([5, 1])
Epoca MLP optimizador ADAM:  (5, 20.85460662841797)
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Siz

torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])


torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([5, 1])
Epoca MLP optimizador ADAM:  (16, 13.656963348388672)
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.S

torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([5, 1])
Epoca MLP optimizador ADAM:  (22, 9.570182800292969)
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Si

torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])


torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([5, 1])
Epoca MLP optimizador ADAM:  (33, 9.54515552520752)
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Siz

torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([5, 1])
Epoca MLP optimizador ADAM:  (39, 6.557735443115234)
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Si

torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])


[tensor(23.0420, grad_fn=<NllLossBackward>), tensor(22.5195, grad_fn=<NllLossBackward>), tensor(23.1073, grad_fn=<NllLossBackward>), tensor(22.5017, grad_fn=<NllLossBackward>), tensor(20.7214, grad_fn=<NllLossBackward>), tensor(20.8546, grad_fn=<NllLossBackward>), tensor(18.3327, grad_fn=<NllLossBackward>), tensor(20.5901, grad_fn=<NllLossBackward>), tensor(18.0894, grad_fn=<NllLossBackward>), tensor(14.5715, grad_fn=<NllLossBackward>), tensor(15.6055, grad_fn=<NllLossBackward>), tensor(18.8545, grad_fn=<NllLossBackward>), tensor(14.4159, grad_fn=<NllLossBackward>), tensor(12.5809, grad_fn=<NllLossBackward>), tensor(16.4143, grad_fn=<NllLossBackward>), tensor(14.0110, grad_fn=<NllLossBackward>), tensor(13.6570, grad_fn=<NllLossBackward>), tensor(13.6080, grad_fn=<NllLossBackward>), tensor(16.3272, grad_fn=<NllLossBackward>), tensor(7.5853, grad_fn=<NllLossBackward>), tensor(12.2061, grad_fn=<NllLossBackward>), tensor(9.3795, grad_fn=<NllLossBackward>), tensor(9.5702, grad_fn=<NllLossBa

In [133]:
for batch_idx, (XX,YY) in enumerate(testloader):
    error = 0
    out = prueba2(XX)
    for i in range(cantTest):
        valor, outNet = torch.max(out[i],0)
        print("Salida de la red: ",outNet)
        #print("Salida de la red: ",out[i].data)
        out[i]
        print("La posta: ",YY[i].item())
        time.sleep(2)
        #if numpy.sign(out[i].item()) != YY[i].item():
            #error = error + 1

Salida de la red:  tensor(44)
La posta:  44
Salida de la red:  tensor(90)
La posta:  99
Salida de la red:  tensor(99)
La posta:  81
Salida de la red:  tensor(101)
La posta:  11
Salida de la red:  tensor(70)
La posta:  58
Salida de la red:  tensor(28)
La posta:  16
Salida de la red:  tensor(69)
La posta:  69
Salida de la red:  tensor(28)
La posta:  100
Salida de la red:  tensor(68)
La posta:  26
Salida de la red:  tensor(22)
La posta:  22
Salida de la red:  tensor(98)
La posta:  80
Salida de la red:  tensor(35)
La posta:  50
Salida de la red:  tensor(88)
La posta:  43
Salida de la red:  tensor(29)
La posta:  95
Salida de la red:  tensor(25)
La posta:  52
Salida de la red:  tensor(68)
La posta:  68


KeyboardInterrupt: 

In [119]:
def create_datasets(batch_size):

    # percentage of training set to use as validation
    valid_size = 0.1

    # convert data to torch.FloatTensor
    transform = transforms.ToTensor()

    # choose the training and test datasets
    train_data = tensorProm[0:5,:]

    test_data = tensorProm[6,:]

    # obtain training indices that will be used for validation
    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]
    
    # define samplers for obtaining training and validation batches
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    
    # load training data in batches
    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=train_sampler,
                                               num_workers=0)
    
    # load validation data in batches
    valid_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=valid_sampler,
                                               num_workers=0)
    
    # load test data in batches
    test_loader = torch.utils.data.DataLoader(test_data,
                                              batch_size=batch_size,
                                              num_workers=0)
    
    return train_loader, test_loader, valid_loader

In [9]:
(train_loader, test_loader, valid_loader)=create_datasets(1)
print(train_loader)
print(test_loader)
print(valid_loader)


#Literalmente 45 minutos para encontrar esta linea:
print(next(iter(test_loader)))

tensor([0.0941], dtype=torch.float64)
